# Agent Sandbox Custom Configuration Guide

## Description

Agent Sandbox supports extended configuration via `x-custom-config`, allowing users to create sandboxes with advanced features such as custom images, environment variables, resource limits, and health checks.

### Key Features

- **Custom Images**: Support for custom images from enterprise or personal image registries
- **Environment Variables**: Support for injecting custom environment variables at sandbox startup
- **Resource Limits**: Support for configuring CPU and memory resource limits
- **Port Configuration**: Support for configuring multiple container ports
- **Health Checks**: Support for HTTP health check probes
- **Startup Commands**: Support for custom container startup commands and arguments

### Use Cases

- AI Agents requiring specific runtime environments
- Code execution environments with pre-installed dependencies
- Applications requiring precise resource allocation control
- Services requiring custom health checks

## Install Dependencies

In [ ]:
%pip install e2b_code_interpreter

## x-custom-config Data Structure

When creating a sandbox, you can pass custom configuration through the `x-custom-config` key in the `metadata` field.

### Complete Data Structure

```json
{
  "image": "ccr.ccs.tencentyun.com/my-repo/my-image:v1.0",
  "imageRegistryType": "enterprise",
  "imageDigest": "sha256:abc123...",
  "command": ["/bin/sh", "-c"],
  "args": ["./start.sh"],
  "env": [
    {"name": "NODE_ENV", "value": "production"},
    {"name": "DEBUG", "value": "true"}
  ],
  "ports": [
    {"name": "http", "port": 8080, "protocol": "TCP"},
    {"name": "metrics", "port": 9090, "protocol": "TCP"}
  ],
  "resources": {
    "cpu": "2",
    "memory": "4Gi"
  },
  "probe": {
    "httpGet": {
      "path": "/health",
      "port": 8080,
      "scheme": "HTTP"
    },
    "readyTimeoutMs": 30000,
    "probeTimeoutMs": 5000,
    "probePeriodMs": 10000,
    "successThreshold": 1,
    "failureThreshold": 3
  }
}
```

### Field Descriptions

| Field | Type | Required | Description |
|-------|------|----------|-------------|
| `image` | string | No | Custom image address |
| `imageRegistryType` | string | No | Image registry type: `enterprise` or `personal` |
| `imageDigest` | string | No | Image digest (sha256), only returned in response |
| `command` | []string | No | Container startup command (overrides image ENTRYPOINT) |
| `args` | []string | No | Container startup arguments (overrides image CMD) |
| `env` | []EnvVar | No | Environment variable list |
| `ports` | []PortConfig | No | Port configuration list |
| `resources` | ResourceConfig | No | Resource configuration (CPU/Memory) |
| `probe` | ProbeConfig | No | Health check configuration |

## Environment Setup

In [ ]:
import os
import json
from e2b_code_interpreter import Sandbox

# Get API Key and available domain from the Agent Sandbox console
# Can be set via environment variables or modified directly here
if not os.getenv("E2B_DOMAIN"):
    os.environ["E2B_DOMAIN"] = "ap-guangzhou.tencentags.com"
if not os.getenv("E2B_API_KEY"):
    os.environ["E2B_API_KEY"] = "your-api-key"

## Create Sandbox with Custom Image

You can specify custom image address and registry type using `x-custom-config`.

In [ ]:
# Define custom image configuration
custom_image_config = {
    "image": "ccr.ccs.tencentyun.com/my-repo/my-app:v1.0",
    "imageRegistryType": "personal"  # Personal image registry
}

# Pass x-custom-config via metadata when creating sandbox
sandbox = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(custom_image_config)
    }
)

print(f"Sandbox ID: {sandbox.sandbox_id}")
print(f"Sandbox Info: {sandbox.get_info()}")

## Execute Commands


In [ ]:
# Create sandbox with environment variables
sandbox = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(env_config)
    }
)

print(f"Sandbox Info: {sandbox.get_info()}")

# Run command
res = sandbox.commands.run(user="root", cmd="echo 'Hello World'")
print(res)

# Cleanup
sandbox.kill()

## Configure Resource Limits

You can configure CPU and memory limits for the sandbox using the `resources` field.

In [ ]:
# Define resource limits configuration
resource_config = {
    "resources": {
        "cpu": "2",      # 2 CPU cores
        "memory": "4Gi"  # 4GB memory
    }
}

# Create sandbox with resource limits
sandbox_with_resources = Sandbox.create(
    template="",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(resource_config)
    }
)

print(f"Sandbox created with resource config: CPU={resource_config['resources']['cpu']}, Memory={resource_config['resources']['memory']}")

# Cleanup
sandbox_with_resources.kill()

## Configure Ports

You can configure container exposed ports using the `ports` field.

In [ ]:
# Define port configuration
port_config = {
    "ports": [
        {"name": "http", "port": 8080, "protocol": "TCP"},
        {"name": "metrics", "port": 9090, "protocol": "TCP"},
        {"name": "debug", "port": 5005, "protocol": "TCP"}
    ]
}

# Create sandbox with port configuration
sandbox_with_ports = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(port_config)
    }
)

# Get access address for each port
for port_info in port_config["ports"]:
    host = sandbox_with_ports.get_host(port_info["port"])
    print(f"{port_info['name']} port ({port_info['port']}): {host}")

# Cleanup
sandbox_with_ports.kill()

## Configure Health Checks

You can configure HTTP health check probes using the `probe` field to monitor the health status of services in the sandbox.

In [ ]:
# Define health check configuration
probe_config = {
    "probe": {
        "httpGet": {
            "path": "/health",
            "port": 8080,
            "scheme": "HTTP"
        },
        "readyTimeoutMs": 30000,    # Ready timeout: 30 seconds
        "probeTimeoutMs": 5000,     # Probe timeout: 5 seconds
        "probePeriodMs": 10000,     # Probe check period: 10 seconds
        "successThreshold": 1,      # Success threshold
        "failureThreshold": 3       # Failure threshold
    }
}

# Create sandbox with health check
sandbox_with_probe = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(probe_config)
    }
)

print("Sandbox created with health check configured:")
print(f"  - Check path: {probe_config['probe']['httpGet']['path']}")
print(f"  - Check port: {probe_config['probe']['httpGet']['port']}")
print(f"  - Ready timeout: {probe_config['probe']['readyTimeoutMs']}ms")
print(f"  - Failure threshold: {probe_config['probe']['failureThreshold']}")

# Cleanup
sandbox_with_probe.kill()

## Configure Custom Startup Commands

You can override the image's default ENTRYPOINT and CMD using the `command` and `args` fields.

In [ ]:
# Define custom startup command configuration
command_config = {
    "command": ["/bin/sh", "-c"],
    "args": ["echo 'Starting custom app...' && ./start.sh"]
}

# Create sandbox with custom startup command
sandbox_with_command = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(command_config)
    }
)

print("Sandbox created with custom startup command:")
print(f"  - Command: {command_config['command']}")
print(f"  - Args: {command_config['args']}")

# Cleanup
sandbox_with_command.kill()

## Comprehensive Configuration Example

The following example demonstrates how to combine multiple configurations to create a fully-featured custom sandbox.

In [ ]:
# Define comprehensive configuration
full_custom_config = {
    # Custom image
    "image": "ccr.ccs.tencentyun.com/my-repo/my-app:v1.0",
    "imageRegistryType": "enterprise",
    
    # Startup command
    "command": ["/bin/sh", "-c"],
    "args": ["./start.sh"],
    
    # Environment variables
    "env": [
        {"name": "NODE_ENV", "value": "production"},
        {"name": "LOG_LEVEL", "value": "info"},
        {"name": "MAX_WORKERS", "value": "4"}
    ],
    
    # Port configuration
    "ports": [
        {"name": "http", "port": 8080, "protocol": "TCP"},
        {"name": "metrics", "port": 9090, "protocol": "TCP"}
    ],
    
    # Resource limits
    "resources": {
        "cpu": "2",
        "memory": "4Gi"
    },
    
    # Health check
    "probe": {
        "httpGet": {
            "path": "/health",
            "port": 8080,
            "scheme": "HTTP"
        },
        "readyTimeoutMs": 30000,
        "probeTimeoutMs": 5000,
        "probePeriodMs": 10000,
        "successThreshold": 1,
        "failureThreshold": 3
    }
}

# Print configuration info
print("Full custom configuration:")
print(json.dumps(full_custom_config, indent=2, ensure_ascii=False))

In [ ]:
# Create sandbox using comprehensive configuration
sandbox_full = Sandbox.create(
    template="your-sandboxtool-name",
    timeout=3600,
    metadata={
        "x-custom-config": json.dumps(full_custom_config)
    }
)

print(f"Sandbox created successfully!")
print(f"Sandbox ID: {sandbox_full.sandbox_id}")

# Get sandbox info
info = sandbox_full.get_info()
print(f"\nSandbox Info:")
print(f"  - Started at: {info.started_at}")
print(f"  - Ends at: {info.end_at}")

## Query Sandbox to Get Configuration Info

The query API returns `x-custom-config` in the response `metadata`, which also includes an additional `imageDigest` field.

In [ ]:
# Get sandbox list and view configuration info
paginator = Sandbox.list(limit=5)

print("Sandbox list and configuration info:")
for sandbox_info in paginator.next_items():
    print(f"\nSandbox ID: {sandbox_info.sandbox_id}")
    if hasattr(sandbox_info, 'metadata') and sandbox_info.metadata:
        if 'x-custom-config' in sandbox_info.metadata:
            config = json.loads(sandbox_info.metadata['x-custom-config'])
            print(f"  Custom configuration:")
            if 'image' in config:
                print(f"    - Image: {config['image']}")
            if 'imageDigest' in config:
                print(f"    - Image digest: {config['imageDigest']}")
            if 'resources' in config:
                print(f"    - Resources: CPU={config['resources'].get('cpu')}, Memory={config['resources'].get('memory')}")

## Error Handling

When `x-custom-config` format is invalid, the API returns an HTTP 400 error.

In [ ]:
# Example: Handle configuration errors
try:
    # Invalid JSON format
    invalid_sandbox = Sandbox.create(
        template="your-sandboxtool-name",
        timeout=3600,
        metadata={
            "x-custom-config": "invalid json format"
        }
    )
except Exception as e:
    print(f"Creation failed, error: {e}")
    print("\nTip: x-custom-config must be a valid JSON string")

In [ ]:
# Correct approach: Use json.dumps to ensure proper format
def create_custom_sandbox(config: dict, template: str = "your-sandboxtool-name", timeout: int = 3600):
    """
    Create a sandbox with custom configuration
    
    Args:
        config: Custom configuration dictionary
        template: Sandbox template
        timeout: Timeout in seconds
    
    Returns:
        Sandbox instance
    """
    try:
        # Use json.dumps to ensure proper configuration format
        config_str = json.dumps(config)
        
        sandbox = Sandbox.create(
            template=template,
            timeout=timeout,
            metadata={
                "x-custom-config": config_str
            }
        )
        return sandbox
    except Exception as e:
        print(f"Failed to create sandbox: {e}")
        return None

# Use the wrapper function to create sandbox
my_config = {
    "env": [{"name": "MY_VAR", "value": "test"}],
    "resources": {"cpu": "1", "memory": "2Gi"}
}

sandbox = create_custom_sandbox(my_config)
if sandbox:
    print(f"Sandbox created successfully: {sandbox.sandbox_id}")
    sandbox.kill()

## Summary

This tutorial covered how to create custom sandboxes using `x-custom-config` extended configuration. Key topics include:

1. **Custom Images**: Use `image` and `imageRegistryType` to specify custom images
2. **Environment Variables**: Use `env` to configure environment variable lists
3. **Resource Limits**: Use `resources` to configure CPU and memory limits
4. **Port Configuration**: Use `ports` to configure container ports
5. **Health Checks**: Use `probe` to configure HTTP health check probes
6. **Startup Commands**: Use `command` and `args` to customize startup commands

### Best Practices

- Always use `json.dumps()` to convert configuration dictionaries to JSON strings
- Configure resource limits appropriately based on actual requirements